# Assignment 7

### Part A

In [ ]:
import numpy as np
import pandas as pd

Consider the following way of representing a portfolio. Each row in the DataFrame represents one security, with information given about the security's beta, volatility, and amount invested in that security. Note that there is not guaranteed to be 2 securities in the portfolio.

In [ ]:
portfolio = pd.DataFrame(data = {"Beta": [1.2, 1.4], 
                                 "Volatility": [0.25, 0.4],
                                 "Amount Invested": [4000, 6000] })



portfolio

Write the function ```expected_return```, which calculates the expected return of the portfolio ```P```. There is a risk free asset with risk free rate ```rf```, and the market risk premium is ```mkt_rp```. 

expected return: expected_return(portfolio, 0.02, 0.05) should yield 0.086. 

In [ ]:
def expected_return(P, rf, mkt_rp):
    # Write your code here
    total_val = P["Amount Invested"].sum()
    beta = 0
    for index, row in P.iterrows():
        beta += row["Amount Invested"] / total_val * row["Beta"]
    return rf + beta * mkt_rp

Write the function ```volatility```, which calculates the volatility of the portfolio. You can assume that the correlation between all (different) pairs of stocks in the portfolio is ```cor```.

expected return: volatility(portfolio, -0.2) should yield 0.2408318915758459

In [ ]:
def volatility(P, cor):
    # Write your code here
    var = 0
    total_val = P["Amount Invested"].sum()
    for index, row in P.iterrows():
        x = row["Amount Invested"] / total_val
        var += x**2 * row["Volatility"]**2
        for index2, row in P.iterrows():
            x_2 = row["Amount Invested"] / total_val
            if (index != index2):
                var += x * x_2 * P.at[index, "Volatility"] * P.at[index2, "Volatility"] * cor
    arr = np.array(var)
    return np.sqrt(arr)

Suppose there is a risk-free asset with return of ```rf```, the market risk premium is ```mkt_rp```, and the volatility of returns on the market is ```mkt_vol```.

Write the function ```expected_return2```, which creates a portfolio with volatility ```vol``` using (only) the risk free asset and the market portfolio, and calculates the expected return of that portfolio.

expected return: expected_return2(0.02, 0.05, 0.16, 0.2408) should yield 0.09525

In [ ]:
def expected_return2(rf, mkt_rp, mkt_vol, vol):
    # Write your code here
    x = vol / mkt_vol
    return rf + x * mkt_rp

### Part B

Write the function ```CAPM_beta```, which will consume a **filepath** ```return_path```, (formatted as the provided "monthlycsrp_mod7.csv"), and a **filepath** `factors_path` (formatted as the provided "F-F_Research_Data_Factors.CSV"). This function will return a DataFrame formatted like the one supplied. Each row in the returned DataFrame should correspond with 1 CUSIP, its expected return, and its CAPM-beta.

Please note that monthlycsrp_mod7.csv's return data are in decimals. "F-F_Research_Data_Factors.CSV" has data in percentages. More details of "F-F_Research_Data_Factors.CSV" are available on https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/f-f_factors.html. 

Also, note that firm i's CAPM-beta is calculated as Cov(R_i-Rf, R_mkt – Rf)/Var(R_mkt-Rf). 

expected return: it should return output of 3225 rows × 3 columns

In [ ]:
def CAPM_beta(return_path, factors_path):
    returns_df = pd.read_csv(return_path)

    factors_df = pd.read_csv(factors_path)
    factors_df["Mkt-RF"] = factors_df["Mkt-RF"] / 100
    factors_df["RF"] = factors_df["RF"] / 100

    # Convert the "date" column to YearMonth format
    factors_df['date'] = pd.to_datetime(factors_df['date'], format='%Y%m')
    factors_df['year'] = factors_df['date'].dt.year
    factors_df['month'] = factors_df['date'].dt.month

    # Merge return and factors data on year and month
    merged_df = pd.merge(returns_df, factors_df, how='inner', on=['year', 'month'])

    # Calculate excess returns for the market and individual stocks
    merged_df['Rm-Rf'] = merged_df['Mkt-RF'] - merged_df['RF']
    merged_df['Ret-Rf'] = merged_df['RET'] - merged_df['RF']

    # Group by CUSIP and calculate beta for each group
    beta_df = merged_df.groupby('CUSIP').apply(lambda group: 
                                               pd.Series({
                                                   'EXP_RETURN': group['RET'].mean(),
                                                   'BETA': (group['Ret-Rf'].cov(group['Rm-Rf']) / group['Rm-Rf'].var())
                                               })
                                              ).reset_index()

    return beta_df

Write the function ```slope```, which will return the slope of the security market line. This function will take in a **DataFrame** ```data```, whose format is that of the output of ```CAPM_beta```. You may find the function `np.polyfit` useful.

In [ ]:
def slope(data):
    # This should return 0.007981588120535642... with the provided data.
    slope = np.polyfit(data["BETA"], data["EXP_RETURN"], 1)[0]
    return slope